In [8]:
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

# Get data
ticker = "AAPL"  
end_date = datetime.now()
start_date = end_date - timedelta(days=5)

data = yf.download(ticker, start=start_date, end=end_date, interval="1h")
data["VPT"] = np.nan
data["RSI"] = np.nan

# Calculate VPT indicator
data["Price Change"] = data["Close"] - data["Close"].shift(1)
data["Volume Multiplier"] = data["Price Change"] * data["Volume"]
data["VPT"] = data["Volume Multiplier"].cumsum()

# Calculate RSI indicator
delta = data["Close"].diff()
gain, loss = delta.copy(), delta.copy()
gain[gain < 0] = 0
loss[loss > 0] = 0
loss = abs(loss)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
data["RSI"] = 100 - (100 / (1 + rs))

# Get the latest trading day data
latest_data = data.iloc[-1:]

# Determine if VPT and price are in sync (rising or falling)
vpt_trend = 0
if latest_data["VPT"].values[0] > data["VPT"].iloc[-2] and latest_data["Close"].values[0] > data["Close"].iloc[-2]:
    vpt_trend = 2 # Rising in sync
elif latest_data["VPT"].values[0] < data["VPT"].iloc[-2] and latest_data["Close"].values[0] < data["Close"].iloc[-2]:
    vpt_trend = 1 # Falling in sync
else:
    vpt_trend = 0 # No clear trend
    
# Get RSI value
rsi_value = latest_data["RSI"].values[0]

########################################

start_date = end_date - timedelta(days=90)

data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
data["MACD"]    = np.nan
data["Signal"]  = np.nan

# Calculate MACD indicator
exp12 = data["Close"].ewm(span=12, adjust=False).mean()
exp26 = data["Close"].ewm(span=26, adjust=False).mean()
macd = exp12 - exp26
signal = macd.ewm(span=9, adjust=False).mean()
data["MACD"] = macd
data["Signal"] = signal

# Get the latest trading day data
latest_data = data.iloc[-1:]

# Determine the MACD trend
macd_trend = 0
if latest_data["MACD"].values[0] > latest_data["Signal"].values[0]:
    macd_trend = 2 # "Bullish"
elif latest_data["MACD"].values[0] < latest_data["Signal"].values[0]:
    macd_trend = 1 # "Bearish"
else:
    macd_trend = 0 # "Neutral"

print(f"Latest trading day:\nVPT and price trend: {vpt_trend}\nRSI value: {rsi_value}")
print(f"Latest trading day:\nMACD trend: {macd_trend}")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Latest trading day:
VPT and price trend: 2
RSI value: 35.52264555379037
Latest trading day:
MACD trend: 1
